In [11]:
import requests
from tqdm import tqdm
import xml.dom.minidom
    
base_url = "https://service.gdi-sh.de/SH_INSPIREDOWNLOAD_AI_CP_ALKIS?" #Schleswig-Holstein
layer_name = "cp:CadastralParcel"
output_path = r"C:\DVP_Projects\data\resultWfsSchleswig-Holstein.gml"
# total_pages_to_fetch = 49

In [43]:
# Get the max batch size for each page
request_size = requests.get(base_url , params={
    "service": "WFS",
    "request": "getcapabilities"
})
request_size_parse = xml.dom.minidom.parseString(request_size.content)
constraint_elements = request_size_parse.getElementsByTagName('ows:Constraint')
for element in constraint_elements :
    if element.getAttribute("name") == "CountDefault":
        default_value = element.getElementsByTagName("ows:DefaultValue")[0]
        batch_size = int(default_value.firstChild.nodeValue)
print(batch_size)

10000


In [13]:
# Calculate the number of pages for the resquest:
request_hits = requests.get(base_url , params={
    "service": "WFS",
    "version": "2.0.0",
    "request": "GetFeature",
    "typeNames": "cp:CadastralParcel",
    "outputFormat": "application/gml+xml; version=3.2",
    "resultType":"hits"
})
print(request_hits.text)
result_parse = xml.dom.minidom.parseString(request_hits.content)
result_fc = result_parse.getElementsByTagName("wfs:FeatureCollection")
total_hits = int(result_fc[0].getAttribute("numberMatched")) #numberOfFeatures #1.1.0
total_pages = (total_hits + batch_size - 1) // batch_size
print(f"Total features: {total_hits}, " f"Total pages: {total_pages}")

<?xml version="1.0" encoding="utf-8"?>
<wfs:FeatureCollection xmlns:wfs="http://www.opengis.net/wfs/2.0" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation=" http://www.opengis.net/wfs/2.0 http://10.61.159.191/aaa-suite/schema/ogc/wfs/2.0/wfs.xsd" timeStamp="2023-09-13T15:21:47.924+02:00" numberReturned="0" numberMatched="1955198"/>
Total features: 1955198, Total pages: 196


In [ ]:
response = requests.get(base_url , params={
    "service": "WFS",
    "version": "2.0.0",
    "request": "GetFeature",
    "typeNames": "cp:CadastralParcel",
    "outputFormat": "text/xml; subtype=gml/3.2.1", #"application/gml+xml; version=3.2",
    "srsName" : "urn:ogc:def:crs:EPSG::4258",
    "startIndex": "80",
    "count" : "10"
}, stream=False)
# print(response.text)
result_parse = xml.dom.minidom.parseString(response.content)
result_fc = result_parse.getElementsByTagName("wfs:FeatureCollection")
print(result_parse.toxml())

In [ ]:
header = response.text.split("<wfs:member>")[0]
print(header)

In [ ]:
feature_collections = result_parse.getElementsByTagName("wfs:FeatureCollection")
for feature_collection in feature_collections:
    # print(f"feature collection: {feature_collection}")
    # feature_members = feature_collection.childNodes
    feature_members = feature_collection.getElementsByTagName("wfs:member")
    for feature_member in feature_members:
        # output_file.write(feature_member.toxml().encode('utf-8')) # write the element in the outputfile
        print(feature_member.toxml())

Versão Final:

In [42]:
# Open the output file for writing in append mode
output_file = open(output_path, "wb")

# Call GetFeature once to fetch the GML header
params1 = {
    "service": "WFS",
    "version": "2.0.0",
    "request": "GetFeature",
    "typeNames": "cp:CadastralParcel",
    "count":"1", #maxFeatures
    "outputFormat": "application/gml+xml; version=3.2"
}

response_header = requests.get(base_url, params=params1)
header = response_header.text.split("<wfs:member>")[0]

# Write the header in the opened document:
output_file.write(header.encode('utf-8'))

# Loop through the pages and fetch only the elements that represent the actual features (exclude each page header)
# for page_number in tqdm(range(total_pages), desc ="Going through pages"):
start_index = 0
while start_index < total_hits :
    params2 = {
        "service": "WFS",
        "version": "2.0.0",
        "request": "GetFeature",
        "typeNames": layer_name,
        "startIndex": start_index,
        "count": batch_size, #maxFeatures
        "outputFormat": "application/gml+xml; version=3.2",
}
    response = requests.get(base_url, params=params2, stream=True)
    root = xml.dom.minidom.parseString(response.content)
    start_index += batch_size
    
    # Find all featureCollection
    feature_collections = root.getElementsByTagName("wfs:FeatureCollection")
    for feature_collection in feature_collections:
        #print(f"feature collection: {feature_collection}")
        # Find and Append each featureMember element to the output file
        feature_members = feature_collection.getElementsByTagName("wfs:member") #feature_collection.childNodes
        for feature_member in feature_members:
            output_file.write(feature_member.toxml().encode('utf-8')) # write the element in the outputfile
            #print(feature_member.toxml(), file=output_file)
            
# Close the GML file with the closing tag
closing_gml = "\n</wfs:FeatureCollection>"
output_file.write(closing_gml.encode('utf-8'))

# Close the document
output_file.close()